In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-07"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11234,2024-05-07,Libano Divisão 1,11:45,Beirut Club,Sagesse Al Hekmeh Beirut,1.98,1.75,181.5,1.83,1.87,-1.5,1.85,1.88,GjuzE7cL,0.505051,0.571429,0.546448,0.534759,0.076479,190.740,92.529460,0.485108,2.4,1.341641,0.559017,305.55,1.930,0.941409,0.487776,61.0,165.980,67.912128,0.409159,1.8,1.643168,0.912871,138.37,1.640,0.733451,0.447226,55.0,105,101,2.91,1.37,149.350,160.844,0.087204,0.015289,0.011374,0.14,0.028,35.000000,0.000000,0.0,0.000000,0.59,0.118,6.355932,0.000000,0.0,0.000000
11235,2024-05-07,República Tcheca Nbl,13:00,Svitavy,Slavia Prague,7.70,1.07,167.5,1.83,1.83,0.0,0.00,0.00,EcgP1eph,0.129870,0.934579,0.546448,0.546448,0.064450,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,257.666,114.837345,0.445683,1.8,1.643168,0.912871,152.25,3.378,1.333593,0.394788,-6.0,0,87,0.00,1.75,0.000,303.456,1.069126,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,3.20,0.640,0.109375,0.215666,0.3,0.084334
11236,2024-05-07,República Tcheca Nbl,13:00,Decin,Usti n. Labem,1.57,2.31,166.5,1.87,1.79,0.0,0.00,0.00,ziKQ0nZ9,0.636943,0.432900,0.534759,0.558659,0.069843,188.638,75.747403,0.401549,2.4,1.341641,0.559017,286.00,2.236,0.852807,0.381399,20.0,105.556,9.362184,0.088694,1.8,1.643168,0.912871,102.18,1.322,0.107564,0.081365,24.0,88,78,3.25,1.31,195.706,215.378,0.269721,0.030912,NaN,4.94,0.988,0.576923,0.665395,0.7,0.034605,-1.51,-0.302,-4.337748,0.495923,0.4,-0.095923
11237,2024-05-07,Romênia Divisão A,13:00,CSM Oradea,Voluntari,1.39,2.84,161.5,1.86,1.80,0.0,0.00,0.00,0tCih4aC,0.719424,0.352113,0.537634,0.555556,0.071537,120.832,20.325948,0.168217,1.8,1.643168,0.912871,151.69,1.462,0.328436,0.224648,20.0,207.986,68.543278,0.329557,1.8,1.643168,0.912871,153.12,2.608,1.006414,0.385895,7.0,77,87,1.97,1.76,103.500,192.796,0.484778,0.023184,NaN,0.94,0.188,2.074468,0.816257,0.9,0.083743,-1.38,-0.276,-6.666667,0.527037,0.5,-0.027037
11238,2024-05-07,Itália Série A2,15:30,Udine,JuVi Cremona,1.17,4.71,154.5,1.85,1.85,0.0,0.00,0.00,4pwHrqIO,0.854701,0.212314,0.540541,0.540541,0.067015,115.202,43.081306,0.373963,1.2,1.643168,1.369306,114.00,1.594,0.721166,0.452425,-14.0,185.826,83.165015,0.447542,0.6,1.341641,2.236068,274.30,2.874,1.305002,0.454072,-69.0,95,65,1.20,4.22,96.594,233.892,0.851414,0.000000,NaN,-0.23,-0.046,-3.695652,0.782515,0.8,0.017485,-3.38,-0.676,-5.488166,0.415867,0.3,-0.115867
11239,2024-05-07,Israel Liga Leumit,13:30,Safed,Ramat Hasharon,1.54,2.37,160.5,1.86,1.84,0.0,0.00,0.00,zkLWaTUq,0.649351,0.421941,0.537634,0.543478,0.071292,177.344,58.185443,0.328094,0.6,1.341641,2.236068,186.20,2.208,0.607882,0.275309,-55.0,187.890,67.509277,0.359302,1.2,1.643168,1.369306,104.58,2.538,0.672622,0.265020,-68.0,76,63,2.45,1.66,160.464,208.270,0.300204,0.007644,NaN,-2.85,-0.570,-0.947368,0.000000,0.0,0.000000,-2.38,-0.476,-2.878151,0.378762,0.2,-0.178762
11240,2024-05-07,Itália Série A2,15:30,Forli,Vigevano,1.12,5.56,153.5,1.84,1.86,0.0,0.00,0.00,t8NJMtv8,0.892857,0.179856,0.543478,0.537634,0.072713,105.546,8.369611,0.079298,2.4,1.341641,0.559017,102.66,1.348,0.146185,0.108446,30.0,228.742,62.035428,0.271203,0.6,1.341641,2.236068,317.37,3.182,1.024632,0.322009,-48.0,87,71,1.18,4.47,97.716,270.228,0.939986,0.007644,NaN,-1.45,-0.290,-0.413793,0.775340,0.8,0.024660,-0.78,-0.156,-29.230769,0.266552,0.3,0.033

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
11242,15:30,Itália Série A2,Basket Torino,Trieste,1.61,Back Home
11252,08:00,Taiwan Liga T1,Taiwan Beer Leopards,Kaohsiung Aquas,1.80,Back Home
11260,13:15,Rússia Liga Vtb United,Zenit Petersburg,CSKA Moscow,1.83,Back Home
